In [0]:
from __future__ import absolute_import, division
import json
import os
import time
from keras.utils import Sequence
import numpy as np
import pandas as pd
import gensim
from tqdm import tqdm
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.stem.lancaster import LancasterStemmer
lc = LancasterStemmer()
from nltk.stem import SnowballStemmer
sb = SnowballStemmer("english")
import gc
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.layers.wrappers import TimeDistributed

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate, LSTM, GRU
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

import sys
from os.path import dirname
#sys.path.append(dirname(dirname(__file__)))
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

import spacy

In [0]:
from keras.utils import Sequence
import os
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

## loading data

In [0]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Squad_train_dataframe.csv')

In [67]:
df.head()

,Unnamed: 0,index,question,context,text,answer_start,c_id,question_length,context_length,answer_length
0,0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0,6,109,4
1,1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0,11,109,3
2,2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0,11,109,1
3,3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0,9,109,2
4,4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0,7,109,2


In [68]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

### Word Level

tokenizer_word=Tokenizer()
tokenizer_word.fit_on_texts(df['question'] + df['context'])
tokenizer_question=tokenizer_word.texts_to_sequences(df['question'])
question_word_padded = pad_sequences(list(tokenizer_question), maxlen=12, padding='post')

# tokenizer_context_word=Tokenizer()
# tokenizer_context_word.fit_on_texts(df['context'])
tokenizer_context=tokenizer_word.texts_to_sequences(df['context'])
context_word_padded = pad_sequences(list(tokenizer_context), maxlen=120, padding='post')

word_index = tokenizer_word.word_index
print("dictionary size: ", len(word_index))

# print(tokenizer_context_word.word_counts)
# print(tokenizer_context_word.document_count)
# print(tokenizer_context_word.word_index)
# print(tokenizer_question_word.word_docs)

dictionary size:  94993


In [69]:
### Char Level

tokenizer_char=Tokenizer(char_level=True)
tokenizer_char.fit_on_texts(df['question'] + df['context'])
tokenizer_question=tokenizer_char.texts_to_sequences(df['question'])
question_char_padded = pad_sequences(list(tokenizer_question), maxlen=40, padding='post')


tokenizer_context=tokenizer_char.texts_to_sequences(df['context'])
context_char_padded = pad_sequences(list(tokenizer_context), maxlen=350, padding='post')

char_index = tokenizer_char.word_index
print("dictionary size: ", len(char_index))

dictionary size:  1266


## Pretrained Layers

In [70]:
#load embeddings
#https://www.kaggle.com/vsmolyakov/keras-cnn-with-fasttext-embeddings

print('loading fasttext word embeddings...')
embeddings_index_fasttext = {}
f = open('/content/drive/My Drive/Colab Notebooks/wiki-news-300d-1M.vec', encoding='utf-8')
for line in f:
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index_fasttext[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index_fasttext))

loading fasttext word embeddings...
found 1000000 word vectors


In [0]:
## https://keras.io/preprocessing/text/
embeddings_index_glove = {}
f = open(r'/content/drive/My Drive/Colab Notebooks/glove.840B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = ''.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index_glove[word] = coefs
f.close()

In [72]:
print('found %s word vectors' % len(embeddings_index_glove))

found 2195893 word vectors


In [0]:
embedding_matrix_glove = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index_glove.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_glove[i] = embedding_vector

In [74]:
#embedding matrix
## https://www.kaggle.com/vsmolyakov/keras-cnn-with-fasttext-embeddings

print('preparing fasttext embedding matrix...')
words_not_found = []
nb_words =  len(char_index)
embedding_matrix_fasttext = np.zeros((nb_words, 300))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index_fasttext.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_fasttext[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix_fasttext, axis=1) == 0))

preparing fasttext embedding matrix...
number of null word embeddings: 6


## Metric

In [0]:
##  https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/scripts/accuracy_metric.py

from keras import backend as K


def accuracy(y_true, y_pred):

    def calculate_accuracy(true_and_pred):
        y_true, y_pred_start, y_pred_end = true_and_pred

        start_probability = y_pred_start[K.cast(y_true[0], dtype='int32')]
        end_probability = y_pred_end[K.cast(y_true[1], dtype='int32')]
        return (start_probability + end_probability) / 2.0

    y_true = K.squeeze(y_true, axis=1)
    y_pred_start = y_pred[:, 0, :]
    y_pred_end = y_pred[:, 1, :]
    accuracy = K.map_fn(calculate_accuracy, (y_true, y_pred_start, y_pred_end), dtype='float32')
    return K.mean(accuracy, axis=0)

## Loss Function

In [0]:
## https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/scripts/loss_function.py

def negative_avg_log_error(y_true, y_pred):

    def sum_of_log_probabilities(true_and_pred):
        y_true, y_pred_start, y_pred_end = true_and_pred

        start_probability = y_pred_start[K.cast(y_true[0], dtype='int32')]
        end_probability = y_pred_end[K.cast(y_true[1], dtype='int32')]
        return K.log(start_probability) + K.log(end_probability)

    y_true = K.squeeze(y_true, axis=1)
    y_pred_start = y_pred[:, 0, :]
    y_pred_end = y_pred[:, 1, :]
    batch_probability_sum = K.map_fn(sum_of_log_probabilities, (y_true, y_pred_start, y_pred_end), dtype='float32')
    return -K.mean(batch_probability_sum, axis=0)

## Multimodel GPU

In [0]:
## https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/scripts/multi_gpu_model.py

from keras import Model
from keras.utils import multi_gpu_model
from keras.engine.topology import Layer
from keras.layers.advanced_activations import Softmax
from keras.layers import TimeDistributed, Dense
from keras.activations import linear
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate, LSTM, GRU
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model


class ModelMGPU(Model):
    def __init__(self, ser_model, gpus=None):
        pmodel = multi_gpu_model(ser_model, gpus)
        self.__dict__.update(pmodel.__dict__)
        self._smodel = ser_model

    def __getattribute__(self, attrname):
        '''Override load and save methods to be used from the serial-model. The
        serial-model holds references to the weights in the multi-gpu model.
        '''
        # return Model.__getattribute__(self, attrname)
        if 'load' in attrname or 'save' in attrname:
            return getattr(self._smodel, attrname)

        return super(ModelMGPU, self).__getattribute__(attrname)

## Span Begin

In [0]:
## https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/layers/span_begin.py
class SpanBegin(Layer):

    def __init__(self, **kwargs):
        super(SpanBegin, self).__init__(**kwargs)

    def build(self, input_shape):
        last_dim = input_shape[0][-1] + input_shape[1][-1]
        input_shape_dense_1 = input_shape[0][:-1] + (last_dim, )
        self.dense_1 = Dense(units=1)
        self.dense_1.build(input_shape_dense_1)
        self.trainable_weights = self.dense_1.trainable_weights
        super(SpanBegin, self).build(input_shape)

    def call(self, inputs):
        merged_context, modeled_passage = inputs
        span_begin_input = K.concatenate([merged_context, modeled_passage])
        span_begin_weights = TimeDistributed(self.dense_1)(span_begin_input)
        span_begin_probabilities = Softmax()(K.squeeze(span_begin_weights, axis=-1))
        return span_begin_probabilities

    def compute_output_shape(self, input_shape):
        merged_context_shape, _ = input_shape
        return merged_context_shape[:-1]

    def get_config(self):
        config = super().get_config()
        return config

## Span End

In [0]:
## https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/layers/span_end.py

class SpanEnd(Layer):

    def __init__(self, **kwargs):
        super(SpanEnd, self).__init__(**kwargs)

    def build(self, input_shape):
        emdim = input_shape[0][-1] // 2
        input_shape_bilstm_1 = input_shape[0][:-1] + (emdim*14, )
        self.bilstm_1 = Bidirectional(LSTM(emdim, return_sequences=True))
        self.bilstm_1.build(input_shape_bilstm_1)
        input_shape_dense_1 = input_shape[0][:-1] + (emdim*10, )
        self.dense_1 = Dense(units=1)
        self.dense_1.build(input_shape_dense_1)
        self.trainable_weights = self.bilstm_1.trainable_weights + self.dense_1.trainable_weights
        super(SpanEnd, self).build(input_shape)

    def call(self, inputs):
        encoded_passage, merged_context, modeled_passage, span_begin_probabilities = inputs
        weighted_sum = K.sum(K.expand_dims(span_begin_probabilities, axis=-1) * modeled_passage, -2)
        passage_weighted_by_predicted_span = K.expand_dims(weighted_sum, axis=1)
        tile_shape = K.concatenate([[1], [K.shape(encoded_passage)[1]], [1]], axis=0)
        passage_weighted_by_predicted_span = K.tile(passage_weighted_by_predicted_span, tile_shape)
        multiply1 = modeled_passage * passage_weighted_by_predicted_span
        span_end_representation = K.concatenate(
            [merged_context, modeled_passage, passage_weighted_by_predicted_span, multiply1])

        span_end_representation = self.bilstm_1(span_end_representation)

        span_end_input = K.concatenate([merged_context, span_end_representation])

        span_end_weights = TimeDistributed(self.dense_1)(span_end_input)

        span_end_probabilities = Softmax()(K.squeeze(span_end_weights, axis=-1))
        return span_end_probabilities

    def compute_output_shape(self, input_shape):
        _, merged_context_shape, _, _ = input_shape
        return merged_context_shape[:-1]

    def get_config(self):
        config = super().get_config()
        return config

## Similarity Layer

In [0]:
## https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/layers/similarity_layer.py


class Similarity(Layer):

    def __init__(self, **kwargs):
        super(Similarity, self).__init__(**kwargs)

    def compute_similarity(self, repeated_context_vectors, repeated_query_vectors):
        element_wise_multiply = repeated_context_vectors * repeated_query_vectors
        concatenated_tensor = K.concatenate(
            [repeated_context_vectors, repeated_query_vectors, element_wise_multiply], axis=-1)
        dot_product = K.squeeze(K.dot(concatenated_tensor, self.kernel), axis=-1)
        return linear(dot_product + self.bias)

    def build(self, input_shape):
        word_vector_dim = input_shape[0][-1]
        weight_vector_dim = word_vector_dim * 3
        self.kernel = self.add_weight(name='similarity_weight',
                                      shape=(weight_vector_dim, 1),
                                      initializer='uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='similarity_bias',
                                    shape=(),
                                    initializer='ones',
                                    trainable=True)
        super(Similarity, self).build(input_shape)

    def call(self, inputs):
        context_vectors, query_vectors = inputs
        num_context_words = K.shape(context_vectors)[1]
        num_query_words = K.shape(query_vectors)[1]
        context_dim_repeat = K.concatenate([[1, 1], [num_query_words], [1]], 0)
        query_dim_repeat = K.concatenate([[1], [num_context_words], [1, 1]], 0)
        repeated_context_vectors = K.tile(K.expand_dims(context_vectors, axis=2), context_dim_repeat)
        repeated_query_vectors = K.tile(K.expand_dims(query_vectors, axis=1), query_dim_repeat)
        similarity_matrix = self.compute_similarity(repeated_context_vectors, repeated_query_vectors)
        return similarity_matrix

    def compute_output_shape(self, input_shape):
        batch_size = input_shape[0][0]
        num_context_words = input_shape[0][1]
        num_query_words = input_shape[1][1]
        return (batch_size, num_context_words, num_query_words)

    def get_config(self):
        config = super().get_config()
        return config

## Query to Context Attention

In [0]:
## https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/layers/query_to_context.py

class Q2CAttention(Layer):

    def __init__(self, **kwargs):
        super(Q2CAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        super(Q2CAttention, self).build(input_shape)

    def call(self, inputs):
        similarity_matrix, encoded_context = inputs
        max_similarity = K.max(similarity_matrix, axis=-1)
        # by default, axis = -1 in Softmax
        context_to_query_attention = Softmax()(max_similarity)
        weighted_sum = K.sum(K.expand_dims(context_to_query_attention, axis=-1) * encoded_context, -2)
        expanded_weighted_sum = K.expand_dims(weighted_sum, 1)
        num_of_repeatations = K.shape(encoded_context)[1]
        return K.tile(expanded_weighted_sum, [1, num_of_repeatations, 1])

    def compute_output_shape(self, input_shape):
        similarity_matrix_shape, encoded_context_shape = input_shape
        return similarity_matrix_shape[:-1] + encoded_context_shape[-1:]

    def get_config(self):
        config = super().get_config()
        return config

## Context to Query Attention

In [0]:
##  https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/layers/context_to_query.py

class C2QAttention(Layer):

    def __init__(self, **kwargs):
        super(C2QAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        super(C2QAttention, self).build(input_shape)

    def call(self, inputs):
        similarity_matrix, encoded_question = inputs
        context_to_query_attention = Softmax(axis=-1)(similarity_matrix)
        encoded_question = K.expand_dims(encoded_question, axis=1)
        return K.sum(K.expand_dims(context_to_query_attention, axis=-1) * encoded_question, -2)

    def compute_output_shape(self, input_shape):
        similarity_matrix_shape, encoded_question_shape = input_shape
        return similarity_matrix_shape[:-1] + encoded_question_shape[-1:]

    def get_config(self):
        config = super().get_config()
        return config

## Merged Context

In [0]:
## https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/layers/merged_context.py

class MergedContext(Layer):

    def __init__(self, **kwargs):
        super(MergedContext, self).__init__(**kwargs)

    def build(self, input_shape):
        super(MergedContext, self).build(input_shape)

    def call(self, inputs):
        encoded_context, context_to_query_attention, query_to_context_attention = inputs
        element_wise_multiply1 = encoded_context * context_to_query_attention
        element_wise_multiply2 = encoded_context * query_to_context_attention
        concatenated_tensor = K.concatenate(
            [encoded_context, context_to_query_attention, element_wise_multiply1, element_wise_multiply2], axis=-1)
        return concatenated_tensor

    def compute_output_shape(self, input_shape):
        encoded_context_shape, _, _ = input_shape
        return encoded_context_shape[:-1] + (encoded_context_shape[-1] * 4, )

    def get_config(self):
        config = super().get_config()
        return config

## Highway Layer

In [0]:
## https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/layers/highway_layer.py

from keras.layers import Dense, Activation, Multiply, Add, Lambda
from keras.initializers import Constant


class Highway(Layer):

    activation = None
    transform_gate_bias = None

    def __init__(self, activation='relu', transform_gate_bias=-1, **kwargs):
        self.activation = activation
        self.transform_gate_bias = transform_gate_bias
        super(Highway, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        dim = input_shape[-1]
        transform_gate_bias_initializer = Constant(self.transform_gate_bias)
        input_shape_dense_1 = input_shape[-1]
        self.dense_1 = Dense(units=dim, bias_initializer=transform_gate_bias_initializer)
        self.dense_1.build(input_shape)
        self.dense_2 = Dense(units=dim)
        self.dense_2.build(input_shape)
        self.trainable_weights = self.dense_1.trainable_weights + self.dense_2.trainable_weights

        super(Highway, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        dim = K.int_shape(x)[-1]
        transform_gate = self.dense_1(x)
        transform_gate = Activation("sigmoid")(transform_gate)
        carry_gate = Lambda(lambda x: 1.0 - x, output_shape=(dim,))(transform_gate)
        transformed_data = self.dense_2(x)
        transformed_data = Activation(self.activation)(transformed_data)
        transformed_gated = Multiply()([transform_gate, transformed_data])
        identity_gated = Multiply()([carry_gate, x])
        value = Add()([transformed_gated, identity_gated])
        return value

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = super().get_config()
        config['activation'] = self.activation
        config['transform_gate_bias'] = self.transform_gate_bias
        return config

## Combine Outputs

In [0]:
## https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/layers/combine_outputs.py

class CombineOutputs(Layer):

    def __init__(self, **kwargs):
        super(CombineOutputs, self).__init__(**kwargs)

    def build(self, input_shape):
        super(CombineOutputs, self).build(input_shape)

    def call(self, inputs):
        span_begin_probabilities, span_end_probabilities = inputs
        return K.stack([span_begin_probabilities, span_end_probabilities], axis = 1)

    def compute_output_shape(self, input_shape):
        number_of_tensors = len(input_shape)
        return input_shape[0][0:1] + (number_of_tensors, ) + input_shape[0][1:]

    def get_config(self):
        config = super().get_config()
        return config

# BiDAF

In [0]:
## https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/models/bidaf.py

from keras.layers import Input, TimeDistributed, LSTM, Bidirectional
from keras.models import Model, load_model
from keras.optimizers import Adadelta
from keras.callbacks import CSVLogger, ModelCheckpoint

import os


class BidirectionalAttentionFlow():

    def __init__(self, emdim, max_passage_length=None, max_query_length=None, num_highway_layers=2, num_decoders=1,
                 encoder_dropout=0, decoder_dropout=0):
        self.emdim = emdim
        self.max_passage_length = max_passage_length
        self.max_query_length = max_query_length

        passage_input = Input(shape=(self.max_passage_length, emdim), dtype='float32', name="passage_input")
        question_input = Input(shape=(self.max_query_length, emdim), dtype='float32', name="question_input")

        question_embedding = question_input
        passage_embedding = passage_input
        for i in range(num_highway_layers):
            highway_layer = Highway(name='highway_{}'.format(i))
            question_layer = TimeDistributed(highway_layer, name=highway_layer.name + "_qtd")
            question_embedding = question_layer(question_embedding)
            passage_layer = TimeDistributed(highway_layer, name=highway_layer.name + "_ptd")
            passage_embedding = passage_layer(passage_embedding)

        encoder_layer = Bidirectional(LSTM(emdim, recurrent_dropout=encoder_dropout,
                                           return_sequences=True), name='bidirectional_encoder')
        encoded_question = encoder_layer(question_embedding)
        encoded_passage = encoder_layer(passage_embedding)

        similarity_matrix = Similarity(name='similarity_layer')([encoded_passage, encoded_question])

        context_to_query_attention = C2QAttention(name='context_to_query_attention')([
            similarity_matrix, encoded_question])
        query_to_context_attention = Q2CAttention(name='query_to_context_attention')([
            similarity_matrix, encoded_passage])

        merged_context = MergedContext(name='merged_context')(
            [encoded_passage, context_to_query_attention, query_to_context_attention])

        modeled_passage = merged_context
        for i in range(num_decoders):
            hidden_layer = Bidirectional(LSTM(emdim, recurrent_dropout=decoder_dropout,
                                              return_sequences=True), name='bidirectional_decoder_{}'.format(i))
            modeled_passage = hidden_layer(modeled_passage)

        span_begin_probabilities = SpanBegin(name='span_begin')([merged_context, modeled_passage])
        span_end_probabilities = SpanEnd(name='span_end')(
            [encoded_passage, merged_context, modeled_passage, span_begin_probabilities])

        output = CombineOutputs(name='combine_outputs')([span_begin_probabilities, span_end_probabilities])

        model = Model([passage_input, question_input], [output])

        model.summary()

        try:
            model = ModelMGPU(model)
        except:
            pass

        adadelta = Adadelta(lr=0.01)
        model.compile(loss=negative_avg_log_error, optimizer=adadelta, metrics=[accuracy])

        self.model = model

    def load_bidaf(self, path):
        custom_objects = {
            'Highway': Highway,
            'Similarity': Similarity,
            'C2QAttention': C2QAttention,
            'Q2CAttention': Q2CAttention,
            'MergedContext': MergedContext,
            'SpanBegin': SpanBegin,
            'SpanEnd': SpanEnd,
            'CombineOutputs': CombineOutputs,
            'negative_avg_log_error': negative_avg_log_error,
            'accuracy': accuracy
        }

        self.model = load_model(path, custom_objects=custom_objects)

    def train_model(self, train_generator, steps_per_epoch=None, epochs=1, validation_generator=None,
                    validation_steps=None, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0,
                    save_history=False, save_model_per_epoch=False):

        saved_items_dir = os.path.join(os.path.dirname(__file__), os.pardir, 'saved_items')
        if not os.path.exists(saved_items_dir):
            os.makedirs(saved_items_dir)

        callbacks = []

        if save_history:
            history_file = os.path.join(saved_items_dir, 'history')
            csv_logger = CSVLogger(history_file, append=True)
            callbacks.append(csv_logger)

        if save_model_per_epoch:
            save_model_file = os.path.join(saved_items_dir, 'bidaf_{epoch:02d}.h5')
            checkpointer = ModelCheckpoint(filepath=save_model_file, verbose=1)
            callbacks.append(checkpointer)

        history = self.model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs,
                                           callbacks=callbacks, validation_data=validation_generator,
                                           validation_steps=validation_steps, workers=workers,
                                           use_multiprocessing=use_multiprocessing, shuffle=shuffle,
                                           initial_epoch=initial_epoch)
        if not save_model_per_epoch:
            self.model.save(os.path.join(saved_items_dir, 'bidaf.h5'))

        return history, self.model

    def predict_ans(self, passage, question, squad_version=1.1, max_span_length=25, do_lowercase=True,
                    return_char_loc=False, return_confidence_score=False):

        if type(passage) == list:
            assert all(type(pas) == str for pas in passage), "Input 'passage' must be of type 'string'"

            passage = [pas.strip() for pas in passage]
            contexts = []
            for pas in passage:
                context_tokens = tokenize(pas, do_lowercase)
                contexts.append(context_tokens)

            if do_lowercase:
                original_passage = [pas.lower() for pas in passage]
            else:
                original_passage = passage

        elif type(passage) == str:
            passage = passage.strip()
            context_tokens = tokenize(passage, do_lowercase)
            contexts = [context_tokens, ]

            if do_lowercase:
                original_passage = [passage.lower(), ]
            else:
                original_passage = [passage, ]

        else:
            raise TypeError("Input 'passage' must be either a 'string' or 'list of strings'")

        assert type(passage) == type(
            question), "Both 'passage' and 'question' must be either 'string' or a 'list of strings'"

        if type(question) == list:
            assert all(type(ques) == str for ques in question), "Input 'question' must be of type 'string'"
            assert len(passage) == len(
                question), "Both lists (passage and question) must contain same number of elements"

            questions = []
            for ques in question:
                question_tokens = tokenize(ques, do_lowercase)
                questions.append(question_tokens)

        elif type(question) == str:
            question_tokens = tokenize(question, do_lowercase)
            questions = [question_tokens, ]

        else:
            raise TypeError("Input 'question' must be either a 'string' or 'list of strings'")

        vectors = MagnitudeVectors(self.emdim).load_vectors()
        context_batch = vectors.query(contexts, self.max_passage_length)
        question_batch = vectors.query(questions, self.max_query_length)

        y = self.model.predict([context_batch, question_batch])
        y_pred_start = y[:, 0, :]
        y_pred_end = y[:, 1, :]

        # clearing the session releases memory by removing the model from memory
        # using this, you will need to load model every time before prediction
        # K.clear_session()

        batch_answer_span = []
        batch_confidence_score = []
        for sample_id in range(len(contexts)):
            answer_span, confidence_score = get_best_span(y_pred_start[sample_id, :], y_pred_end[sample_id, :],
                                                          len(contexts[sample_id]), squad_version, max_span_length)
            batch_answer_span.append(answer_span)
            batch_confidence_score.append(confidence_score)

        answers = []
        for index, answer_span in enumerate(batch_answer_span):
            context_tokens = contexts[index]
            start, end = answer_span[0], answer_span[1]

            # word index to character index mapping
            mapping = get_word_char_loc_mapping(original_passage[index], context_tokens)

            char_loc_start = mapping[start]
            # [1] => char_loc_end is set to point to one more character after the answer
            char_loc_end = mapping[end] + len(context_tokens[end])
            # [1] will help us getting a perfect slice without unnecessary increments/decrements
            ans = original_passage[index][char_loc_start:char_loc_end]

            return_dict = {
                "answer": ans,
            }

            if return_char_loc:
                return_dict["char_loc_start"] = char_loc_start
                return_dict["char_loc_end"] = char_loc_end - 1

            if return_confidence_score:
                return_dict["confidence_score"] = batch_confidence_score[index]

            answers.append(return_dict)

        if type(passage) == list:
            return answers
        else:
            return answers[0]

In [0]:
vocab_size_char = len(char_index) 
vocab_size_word=len(word_index) 


In [88]:
context_char_len=context_char_padded.shape[1]
context_word_len=context_word_padded.shape[1]
question_char_len=question_char_padded.shape[1]
question_word_len=question_word_padded.shape[1]

print(context_char_len, context_word_len, question_char_len, question_word_len)

350 120 40 12


In [0]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return tf.enable_eager_execution(float(intersection) / union)

In [191]:
passage_embedding_fasttext.shape[1]



Dimension(120)

In [0]:
## https://github.com/ParikhKadam/bidaf-keras/blob/master/bidaf/layers/similarity_layer.py


class Similarity(Layer):

    def __init__(self, **kwargs):
        super(Similarity, self).__init__(**kwargs)

    def compute_similarity(self, repeated_context_vectors, repeated_query_vectors):
        element_wise_multiply = repeated_context_vectors * repeated_query_vectors
        concatenated_tensor = np.concatenate(
            [repeated_context_vectors, repeated_query_vectors, element_wise_multiply], axis=-1)
        dot_product = K.squeeze(K.dot(concatenated_tensor, self.kernel), axis=-1)
        return linear(dot_product + self.bias)

    def build(self, input_shape):
        word_vector_dim = input_shape[0][-1]
        weight_vector_dim = word_vector_dim * 3
        self.kernel = self.add_weight(name='similarity_weight',
                                      shape=(weight_vector_dim, 1),
                                      initializer='uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='similarity_bias',
                                    shape=(),
                                    initializer='ones',
                                    trainable=True)
        super(Similarity, self).build(input_shape)

    def call(self, inputs):
        context_vectors, query_vectors = inputs
        num_context_words = context_vectors.shape[1]
        num_query_words = query_vectors.shape[1]
        # num_context_words = K.shape(context_vectors)[1]
        # num_query_words = K.shape(query_vectors)[1]
        context_dim_repeat = np.concatenate([[1, 1], [num_query_words], [1]], 0)
        query_dim_repeat = np.concatenate([[1], [num_context_words], [1, 1]], 0)
        repeated_context_vectors = K.tile(K.expand_dims(context_vectors, axis=2), context_dim_repeat)
        repeated_query_vectors = K.tile(K.expand_dims(query_vectors, axis=1), query_dim_repeat)
        similarity_matrix = self.compute_similarity(repeated_context_vectors, repeated_query_vectors)
        return similarity_matrix

    def compute_output_shape(self, input_shape):
        batch_size = input_shape[0][0]
        num_context_words = input_shape[0][1]
        num_query_words = input_shape[1][1]
        return (batch_size, num_context_words, num_query_words)

    def get_config(self):
        config = super().get_config()
        return config

In [197]:
emdim=300
max_passage_length = 120
max_query_length = 12
num_highway_layers=2
encoder_dropout=0
decoder_dropout=0
num_encoders=2
num_decoders=1
passage_input = Input(shape=(max_passage_length, ))
question_input = Input(shape=(max_query_length,))
passage_embedding_fasttext=Embedding(vocab_size_char, 300, weights=[embedding_matrix_fasttext], input_length=max_passage_length, trainable=False)(passage_input)
passage_embedding_glove= Embedding(vocab_size_word+1, 300, weights=[embedding_matrix_glove], input_length=max_passage_length, trainable=False)(passage_input)
question_embedding_fasttext=Embedding(vocab_size_char, 300, weights=[embedding_matrix_fasttext], input_length=max_query_length, trainable=False)(question_input)
question_embedding_glove=Embedding(vocab_size_word+1, 300, weights=[embedding_matrix_glove], input_length=max_query_length, trainable=False)(question_input)
# question_embedding = concatenate([question_embedding_fasttext, question_embedding_glove ], axis=-1, name='question_embeding')
# passage_embedding = concatenate([passage_embedding_fasttext, passage_embedding_glove ], axis=-1, name='passage_embedding')
# for i in range(num_highway_layers):   
highway_layer = Highway(name='highway_1')
question_layer = TimeDistributed(highway_layer, name=highway_layer.name + "_qtd")
question_embedding = question_layer(question_embedding_fasttext)
passage_layer = TimeDistributed(highway_layer, name=highway_layer.name + "_ptd")
passage_embedding = passage_layer(passage_embedding_fasttext)

print("ran till here 1")
encoder_layer = Bidirectional(LSTM(emdim, recurrent_dropout=encoder_dropout,
                                    return_sequences=True), name='bidirectional_encoder')
encoded_question = encoder_layer(question_embedding)
encoded_passage = encoder_layer(passage_embedding)

similarity_matrix = Similarity(name='similarity_layer')([encoded_passage, encoded_question])
# similarity_matrix=jaccard_similarity(encoded_passage, encoded_question)
context_to_query_attention = C2QAttention(name='context_to_query_attention')([
    similarity_matrix, encoded_question])
query_to_context_attention = Q2CAttention(name='query_to_context_attention')([
    similarity_matrix, encoded_passage])

merged_context = MergedContext(name='merged_context')(
    [encoded_passage, context_to_query_attention, query_to_context_attention])

modeled_passage = merged_context
for i in range(num_decoders):
    hidden_layer = Bidirectional(LSTM(emdim, recurrent_dropout=decoder_dropout,
                                      return_sequences=True), name='bidirectional_decoder_{}'.format(i))
    modeled_passage = hidden_layer(modeled_passage)

span_begin_probabilities = SpanBegin(name='span_begin')([merged_context, modeled_passage])
span_end_probabilities = SpanEnd(name='span_end')(
    [encoded_passage, merged_context, modeled_passage, span_begin_probabilities])

output = CombineOutputs(name='combine_outputs')([span_begin_probabilities, span_end_probabilities])

model = Model([passage_input, question_input], [output])

model.summary()


ran till here 1


TypeError: ignored

In [200]:
encoded_passage.shape

TensorShape([Dimension(None), Dimension(None), Dimension(600)])

In [201]:
encoded_question.shape

TensorShape([Dimension(None), Dimension(None), Dimension(600)])